# Import Libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler


# Loading data

## Install dependencies as needed:

In [ ]:
!pip install kagglehub[pandas-datasets]

In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

file_path = "merged.csv"

df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "vishala28/swat-dataset-secure-water-treatment-system",
  file_path,
)


/tmp/ipython-input-3042965120.py:6: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


100%|██████████| 407M/407M [00:26<00:00, 15.9MB/s]


In [ ]:
print("Dataset Shape:", df.shape)

Dataset Shape: (1441719, 53)


In [ ]:
df.head()

,Timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603,Normal/Attack
0,28/12/2015 10:00:00 AM,2.427057,522.8467,2.0,2,1,262.0161,8.396437,328.6337,2.445391,...,2,1,250.8652,1.649953,189.5988,0.000128,1,1,1,Normal
1,28/12/2015 10:00:01 AM,2.446274,522.8860,2.0,2,1,262.0161,8.396437,328.6337,2.445391,...,2,1,250.8652,1.649953,189.6789,0.000128,1,1,1,Normal
2,28/12/2015 10:00:02 AM,2.489191,522.8467,2.0,2,1,262.0161,8.394514,328.6337,2.442316,...,2,1,250.8812,1.649953,189.6789,0.000128,1,1,1,Normal
3,28/12/2015 10:00:03 AM,2.534350,522.9645,2.0,2,1,262.0161,8.394514,328.6337,2.442316,...,2,1,250.8812,1.649953,189.6148,0.000128,1,1,1,Normal
4,28/12/2015 10:00:04 AM,2.569260,523.4748,2.0,2,1,262.0161,8.394514,328.6337,2.443085,...,2,1,250.8812,1.649953,189.5027,0.000128,1,1,1,Normal


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1441719 entries, 0 to 1441718
Data columns (total 53 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0    Timestamp     1441719 non-null  object 
 1   FIT101         1441719 non-null  float64
 2   LIT101         1441719 non-null  float64
 3    MV101         449919 non-null   float64
 4   P101           1441719 non-null  int64  
 5   P102           1441719 non-null  int64  
 6    AIT201        449919 non-null   float64
 7   AIT202         1441719 non-null  float64
 8   AIT203         1441719 non-null  float64
 9   FIT201         1441719 non-null  float64
 10   MV201         449919 non-null   float64
 11   P201          449919 non-null   float64
 12   P202          449919 non-null   float64
 13  P203           1441719 non-null  int64  
 14   P204          449919 non-null   float64
 15  P205           1441719 non-null  int64  
 16  P206           1441719 non-null  int64  
 17  DPIT301 

# Cleaning Data

## Clean Column Names

In [ ]:
df.columns = df.columns.str.strip()


## Convert Timestamp to Datetime & Sort

In [ ]:
df['Timestamp'] = pd.to_datetime(
    df['Timestamp'].str.strip(),
    format='%d/%m/%Y %I:%M:%S %p',
    errors='coerce'
)

df[df['Timestamp'].isna()]

df = df.sort_values("Timestamp")

## Create Binary Label

In [ ]:
df['Normal/Attack'].value_counts()

,count
Normal/Attack,
Normal,1387098
Attack,54621


In [ ]:
df['label'] = df['Normal/Attack'].apply(lambda x: 0 if x == "Normal" else 1)

df = df.drop(columns=["Normal/Attack"])


Now:

0 = Normal

1 = Attack

## chick missing values

In [ ]:
print("Total Missing Values:", df.isnull().sum().sum())


Total Missing Values: 6942600


In [ ]:
df.isnull().sum()

,0
Timestamp,0
FIT101,0
LIT101,0
MV101,991800
P101,0
P102,0
AIT201,991800
AIT202,0
AIT203,0
FIT201,0


**lets investigate more**

*   frst look it seems that the nulls is in some columns is the same






In [ ]:
nulls_cols=['MV101','AIT201','MV201','P201','P202','P204','MV303']

In [ ]:
df[df['MV101'].isnull()]['label'].value_counts()

,count
label,
0,991800


In [ ]:
df[df['MV101'].isnull()][nulls_cols]

,MV101,AIT201,MV201,P201,P202,P204,MV303
395298,NaN,NaN,NaN,NaN,NaN,NaN,NaN
395299,NaN,NaN,NaN,NaN,NaN,NaN,NaN
395300,NaN,NaN,NaN,NaN,NaN,NaN,NaN
395301,NaN,NaN,NaN,NaN,NaN,NaN,NaN
395302,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
892095,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1387096,NaN,NaN,NaN,NaN,NaN,NaN,NaN
892096,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1387097,NaN,NaN,NaN,NaN,NaN,NaN,NaN




1.   the nulls is when there is no attack this is confusing
2.   the nulls aqure in ['MV101','AIT201','MV201','P201','P202','P204','MV303'] at the same time

*   **lets check the values in this columns if no nulls**



In [ ]:
df[~df['MV101'].isnull()][nulls_cols].describe()

,MV101,AIT201,MV201,P201,P202,P204,MV303
count,449919.000000,449919.000000,449919.000000,449919.000000,449919.0,449919.000000,449919.000000
mean,1.665335,210.297302,1.691584,1.121068,1.0,1.000124,1.023349
std,0.482323,35.157909,0.470611,0.326207,0.0,0.011156,0.181441
min,0.000000,168.033800,0.000000,1.000000,1.0,1.000000,0.000000
25%,1.000000,177.102000,1.000000,1.000000,1.0,1.000000,1.000000
50%,2.000000,193.508100,2.000000,1.000000,1.0,1.000000,1.000000
75%,2.000000,253.845200,2.000000,1.000000,1.0,1.000000,1.000000
max,2.000000,267.719800,2.000000,2.000000,1.0,2.000000,2.000000


7 columns with 991,800 nulls ['MV101','AIT201','MV201','P201','P202','P204','MV303']

All other columns complete

Nulls occur together

They form a large structural block

they occur in ***normal state***

This is almost certainly a dataset merge boundary, not sensor behavior.

In [ ]:
df_clean = df[df['MV101'].notnull()].reset_index(drop=True)


In [ ]:
df_clean.isnull().sum()

,0
Timestamp,0
FIT101,0
LIT101,0
MV101,0
P101,0
P102,0
AIT201,0
AIT202,0
AIT203,0
FIT201,0


In [ ]:
df_clean['label'].value_counts()

,count
label,
0,395298
1,54621


In [ ]:
print(f'the shape of df_cleean {df_clean.shape}')
print(f'the shape of df {df.shape}')
print(f'if we choose the part of data that not have these nulls we will lose >> {df.shape[0]-df_clean.shape[0]}')

the shape of df_cleean (449919, 53)
the shape of df (1441719, 53)
if we choose the part of data that not have these nulls we will lose >> 991800


In [ ]:
df.describe()

,Timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603,label
count,1441719,1.441719e+06,1.441719e+06,449919.000000,1.441719e+06,1.441719e+06,449919.000000,1.441719e+06,1.441719e+06,1.441719e+06,...,1.441719e+06,1441719.0,1.441719e+06,1.441719e+06,1.441719e+06,1.441719e+06,1441719.0,1.441719e+06,1441719.0,1.441719e+06
mean,2015-12-27 06:07:18.868643072,1.806108e+00,5.931200e+02,1.665335,1.730916e+00,1.002168e+00,210.297302,8.431862e+00,3.395658e+02,1.791239e+00,...,1.973482e+00,1.0,2.453681e+02,1.131493e+00,1.856421e+02,1.507692e-02,1.0,1.008294e+00,1.0,3.788602e-02
min,2015-12-22 16:00:00,0.000000e+00,1.206237e+02,0.000000,1.000000e+00,1.000000e+00,168.033800,6.000000e+00,2.853371e+02,0.000000e+00,...,1.000000e+00,1.0,8.891951e+00,0.000000e+00,3.108177e+00,0.000000e+00,1.0,1.000000e+00,1.0,0.000000e+00
25%,2015-12-24 18:18:34.500000,0.000000e+00,5.087943e+02,1.000000,1.000000e+00,1.000000e+00,177.102000,8.357024e+00,3.211484e+02,0.000000e+00,...,2.000000e+00,1.0,2.489906e+02,1.009195e+00,1.877724e+02,0.000000e+00,1.0,1.000000e+00,1.0,0.000000e+00
50%,2015-12-26 20:22:09,2.487269e+00,5.268113e+02,2.000000,2.000000e+00,1.000000e+00,193.508100,8.380095e+00,3.300179e+02,2.444238e+00,...,2.000000e+00,1.0,2.512977e+02,1.105309e+00,1.899353e+02,6.407587e-05,1.0,1.000000e+00,1.0,0.000000e+00
75%,2015-12-29 10:51:28.500000,2.588156e+00,6.957159e+02,2.000000,2.000000e+00,1.000000e+00,253.845200,8.450590e+00,3.341451e+02,2.452440e+00,...,2.000000e+00,1.0,2.541656e+02,1.217441e+00,1.931876e+02,1.281517e-04,1.0,1.000000e+00,1.0,0.000000e+00
max,2016-01-02 14:59:59,2.760145e+00,1.000000e+03,2.000000,2.000000e+00,2.000000e+00,267.719800,8.988273e+00,5.674699e+02,2.826899e+00,...,2.000000e+00,1.0,2.646437e+02,3.668343e+00,2.006376e+02,1.802710e+00,1.0,2.000000e+00,1.0,1.000000e+00
std,NaN,1.153932e+00,1.239325e+02,0.482323,4.434837e-01,4.650650e-02,35.157909,1.181389e-01,4.399908e+01,1.084655e+00,...,1.606709e-01,0.0,3.829254e+01,3.001806e-01,2.961345e+01,1.522567e-01,0.0,9.069441e-02,0.0,1.909207e-01


In [ ]:
df_clean.describe()

,Timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,...,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603,label
count,449919,449919.000000,449919.000000,449919.000000,449919.000000,449919.000000,449919.000000,449919.000000,449919.000000,449919.000000,...,449919.000000,449919.0,449919.000000,449919.000000,449919.000000,449919.000000,449919.0,449919.000000,449919.0,449919.000000
mean,2015-12-31 00:29:46.199655936,1.714346,607.019967,1.665335,1.693251,1.006946,210.297302,8.528535,320.301478,1.702908,...,1.925862,1.0,232.173817,1.013408,174.714484,0.016566,1.0,1.009099,1.0,0.121402
min,2015-12-28 10:00:00,0.000000,189.826300,0.000000,1.000000,1.000000,168.033800,6.000000,285.337100,0.000000,...,1.000000,1.0,9.468726,0.000000,3.140220,0.000000,1.0,1.000000,1.0,0.000000
25%,2015-12-29 17:14:39.500000,0.000000,510.011100,1.000000,1.000000,1.000000,177.102000,8.441618,306.383000,0.000000,...,2.000000,1.0,248.029400,0.816967,186.698900,0.000000,1.0,1.000000,1.0,0.000000
50%,2015-12-31 00:29:19,2.477020,530.422500,2.000000,2.000000,1.000000,193.508100,8.551525,321.661100,2.446673,...,2.000000,1.0,249.295100,0.961138,187.724300,0.000000,1.0,1.000000,1.0,0.000000
75%,2016-01-01 07:45:19.500000,2.577907,727.432100,2.000000,2.000000,1.000000,253.845200,8.610806,331.684200,2.454362,...,2.000000,1.0,250.881200,1.409669,189.118200,0.000128,1.0,1.000000,1.0,0.000000
max,2016-01-02 14:59:59,2.760145,925.032300,2.000000,2.000000,2.000000,267.719800,8.733210,384.465500,2.826899,...,2.000000,1.0,254.341800,1.970333,191.986000,1.802710,1.0,2.000000,1.0,1.000000
std,NaN,1.191716,125.303003,0.482323,0.461145,0.083051,35.157909,0.114844,16.631029,1.130277,...,0.261995,0.0,61.750930,0.412723,47.479809,0.159603,0.0,0.094956,0.0,0.326594


***we see that nulls occure in this 2015-12-22  →  2015-12-2810:00:00***


In [ ]:
df[df['Timestamp']< '2015-12-28 10:00:00'][nulls_cols]

,MV101,AIT201,MV201,P201,P202,P204,MV303
395298,NaN,NaN,NaN,NaN,NaN,NaN,NaN
395299,NaN,NaN,NaN,NaN,NaN,NaN,NaN
395300,NaN,NaN,NaN,NaN,NaN,NaN,NaN
395301,NaN,NaN,NaN,NaN,NaN,NaN,NaN
395302,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
892095,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1387096,NaN,NaN,NaN,NaN,NaN,NaN,NaN
892096,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1387097,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df[df['Timestamp'] > '2015-12-28 10:00:00'][nulls_cols]

,MV101,AIT201,MV201,P201,P202,P204,MV303
1,2.0,262.0161,2.0,1.0,1.0,1.0,1.0
2,2.0,262.0161,2.0,1.0,1.0,1.0,1.0
3,2.0,262.0161,2.0,1.0,1.0,1.0,1.0
4,2.0,262.0161,2.0,1.0,1.0,1.0,1.0
5,2.0,262.0161,2.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...
395293,2.0,168.0979,2.0,2.0,1.0,1.0,1.0
395294,2.0,168.0979,2.0,2.0,1.0,1.0,1.0
395295,2.0,168.0979,2.0,2.0,1.0,1.0,1.0
395296,2.0,168.0979,2.0,2.0,1.0,1.0,1.0


## 📌 Conclusion on Missing Values

During the exploratory data analysis, it was observed that the following seven columns:

- MV101  
- AIT201  
- MV201  
- P201  
- P202  
- P204  
- MV303  

each contain **991,800 missing values**.

By analyzing the `Timestamp` column, it was determined that:

- All missing values occur **before** `2015-12-28 10:00:00`.
- After this timestamp, these columns contain **no missing values**.
- The transition from missing to non-missing values happens abruptly at a single point in time.

This indicates that the missing values are **not caused by random sensor failures or temporary dropouts**, but instead reflect a **structural change in data logging configuration**.

The dataset therefore consists of two distinct regimes:

1. **2015-12-22 → 2015-12-28 10:00:00**  
   These seven sensors were not recorded.

2. **2015-12-28 10:00:00 → 2016-01-02**  
   All sensors were fully recorded.

Since this project focuses on **unsupervised anomaly detection**, maintaining a consistent feature space is essential. Mixing both regimes would introduce artificial distribution shifts and negatively impact model training.

Therefore:

- The earlier time block (before `2015-12-28 10:00:00`) was removed.
- No forward-fill, backward-fill, or zero-imputation was applied.
- The analysis proceeds using only the fully observed time segment.

This approach ensures:
- A consistent sensor configuration  
- A stable statistical distribution  
- Reliable anomaly detection modeling

## we found another data set that has our missing data and i will load and merge with the current and apply the same preprocessing


In [4]:
import kagglehub
import os
ab109316_ts_training_path = kagglehub.dataset_download('ab109316/ts-training')
print('Data source import complete.')

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

df_attack_28_2 = pd.read_csv(ab109316_ts_training_path+"/TS Training/data/anomaly_detection/multivariate/SWAT/SWaT.csv")
df_normal_22_28 = pd.read_csv(ab109316_ts_training_path+"/TS Training/data/anomaly_detection/multivariate/SWAT/SWaT_Dataset_Normal_v0.csv")

df_normal_22_28.columns = df_normal_22_28.columns.str.strip()
df_attack_28_2.columns = df_attack_28_2.columns.str.strip()

df_normal_22_28['Timestamp'] = pd.to_datetime(
    df_normal_22_28['Timestamp'].str.strip(),
    format='%d/%m/%Y %I:%M:%S %p',
    errors='coerce'
)

df_normal_22_28[df_normal_22_28['Timestamp'].isna()]

df_normal_22_28 = df_normal_22_28.sort_values("Timestamp")

df_attack_28_2['Timestamp'] = pd.to_datetime(
    df_attack_28_2['Timestamp'].str.strip(),
    format='%d/%m/%Y %I:%M:%S %p',
    errors='coerce'
)

df_attack_28_2[df_attack_28_2['Timestamp'].isna()]

df_attack_28_2 = df_attack_28_2.sort_values("Timestamp")

Using Colab cache for faster access to the 'ts-training' dataset.
Data source import complete.
/kaggle/input/ts-training/TS Training/data/anomaly_detection/multivariate/A Dataset to Support Research in the Design of Secure Water Treatment Systems (1).pdf
/kaggle/input/ts-training/TS Training/data/anomaly_detection/multivariate/process_1_attack.csv
/kaggle/input/ts-training/TS Training/data/anomaly_detection/multivariate/process_1_normal.csv
/kaggle/input/ts-training/TS Training/data/anomaly_detection/multivariate/SWAT/SWaT_Dataset_Normal_v1.csv
/kaggle/input/ts-training/TS Training/data/anomaly_detection/multivariate/SWAT/SWaT.csv
/kaggle/input/ts-training/TS Training/data/anomaly_detection/multivariate/SWAT/SWaT_Dataset_Normal_v0.csv
/kaggle/input/ts-training/TS Training/data/anomaly_detection/multivariate/SWAT/readme/List_of_attacks_Final.xlsx
/kaggle/input/ts-training/TS Training/data/anomaly_detection/multivariate/SWAT/readme/A Dataset to Support Research in the Design of Secure Wa

In [5]:
df_attack_28_2['label'] = df_attack_28_2['Normal/Attack'].apply(lambda x: 0 if x == "Normal" else 1)
df_normal_22_28['label'] = df_normal_22_28['Normal/Attack'].apply(lambda x: 0 if x == "Normal" else 1)



In [6]:
df_attack_28_2 = df_attack_28_2.drop(columns=["Normal/Attack"])
df_normal_22_28 = df_normal_22_28.drop(columns=["Normal/Attack"])


In [7]:
df_full = pd.concat([df_normal_22_28, df_attack_28_2])

In [8]:
df_full.isna().sum()

,0
Timestamp,0
FIT101,0
LIT101,0
MV101,0
P101,0
P102,0
AIT201,0
AIT202,0
AIT203,0
FIT201,0


In [10]:
#df_final = pd.concat([df_full, df_clean]).drop_duplicates()
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Index: 946719 entries, 0 to 449918
Data columns (total 53 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   Timestamp  946719 non-null  datetime64[ns]
 1   FIT101     946719 non-null  float64       
 2   LIT101     946719 non-null  float64       
 3   MV101      946719 non-null  int64         
 4   P101       946719 non-null  int64         
 5   P102       946719 non-null  int64         
 6   AIT201     946719 non-null  float64       
 7   AIT202     946719 non-null  float64       
 8   AIT203     946719 non-null  float64       
 9   FIT201     946719 non-null  float64       
 10  MV201      946719 non-null  int64         
 11  P201       946719 non-null  int64         
 12  P202       946719 non-null  int64         
 13  P203       946719 non-null  int64         
 14  P204       946719 non-null  int64         
 15  P205       946719 non-null  int64         
 16  P206       946719 non-nul

In [11]:
df_full.to_csv('clean_df.csv')

In [12]:
from google.colab import drive
drive.mount('/content/drive')

# Copy the cleaned CSV to the specified Google Drive folder
!cp /content/clean_df.csv /content/drive/MyDrive/DepiProject/clean_df.csv

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df=df_full.copy()

# EDA